# Tugas 2a: Ringkasan teks dengan <i>file</i> kecil menggunakan Titan Text Premier


Di <i>notebook</i> ini, Anda akan menyerap serangkaian kecil teks langsung ke API Amazon Bedrock (menggunakan model Titan Text) dan menginstruksikannya untuk meringkas teks input. Anda dapat menerapkan pendekatan ini untuk meringkas transkrip panggilan, transkrip rapat, buku, artikel, postingan blog, dan konten relevan lainnya ketika panjang teks input berada dalam batas ukuran konteks model.

## Tugas 2a.1: Pengaturan lingkungan

Dalam tugas ini, Anda mengatur lingkungan Anda.

In [ ]:
#Create a service client by name using the default session.
import json
import os
import sys
import warnings

import boto3
import botocore

warnings.filterwarnings('ignore')
module_path = ".."
sys.path.append(os.path.abspath(module_path))

bedrock_client = boto3.client('bedrock-runtime',region_name=os.environ.get("AWS_DEFAULT_REGION", None))



## Tugas 2a.2: Menulis perintah dengan teks untuk diringkas

Dalam tugas ini, Anda akan menggunakan bagian teks pendek dengan token lebih sedikit daripada panjang maksimum yang didukung oleh model fondasi. Sebagai contoh teks input untuk lab ini, Anda menggunakan paragraf dari [postingan blog AWS] (https://aws.amazon.com/jp/blogs/machine-learning/announcing-new-tools-for-building-with-generative-ai-on-aws/) yang mengumumkan Amazon Bedrock.

Perintah dimulai dengan instruksi `Please provide a summary of the following text.`. 

In [ ]:
prompt_data = """

Please provide a summary of the following text:

AWS took all of that feedback from customers, and today we are excited to announce Amazon Bedrock, \
a new service that makes FMs from AI21 Labs, Anthropic, Stability AI, and Amazon accessible via an API. \
Bedrock is the easiest way for customers to build and scale generative AI-based applications using FMs, \
democratizing access for all builders. Bedrock will offer the ability to access a range of powerful FMs \
for text and images—including Amazons Titan FMs, which consist of two new LLMs we’re also announcing \
today—through a scalable, reliable, and secure AWS managed service. With Bedrock’s serverless experience, \
customers can easily find the right model for what they’re trying to get done, get started quickly, privately \
customize FMs with their own data, and easily integrate and deploy them into their applications using the AWS \
tools and capabilities they are familiar with, without having to manage any infrastructure (including integrations \
with Amazon SageMaker ML features like Experiments to test different models and Pipelines to manage their FMs at scale).
"""

## Tugas 2a.3: Membuat isi permintaan dengan parameter perintah dan inferensi 

Dalam tugas ini, Anda akan membuat isi permintaan dengan parameter perintah dan inferensi di atas.

In [ ]:
# request body
body = json.dumps({
    "inputText": prompt_data, 
    "textGenerationConfig":{
        "maxTokenCount":2048,
        "stopSequences":[],
        "temperature":0,
        "topP":0.9
        }
    }) 

## Tugas 2a.4: Menginvokasi model fondasi melalui Boto3

Dalam tugas ini, Anda mengirim permintaan API ke Amazon Bedrock yang menentukan parameter permintaan: `modelId`, `accept`, dan `contentType`. Mengikuti perintah yang diberikan, model fondasi di Amazon Bedrock kemudian merangkum teks input.

### Menyelesaikan Pembuatan <i>Output</i>

Secara <i>default</i>, layanan Amazon Bedrock menghasilkan keseluruhan ringkasan untuk perintah yang diberikan dalam satu <i>output</i>. Layanan ini akan melambat jika <i>output </i>model berisi banyak token. 

In [ ]:
#model configuration and invoke the model
modelId = 'amazon.titan-text-premier-v1:0' # change this to use a different version from the model provider
accept = 'application/json'
contentType = 'application/json'
outputText = "\n"

try:

    response = bedrock_client.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
    response_body = json.loads(response.get('body').read())

    outputText = response_body.get('results')[0].get('outputText')

except botocore.exceptions.ClientError as error:
    
    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubeshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")
        
    else:
        raise error

print(outputText)

### <i>Streaming</i> Pembuatan <i>Output</i>

Selanjutnya, Anda akan mempelajari cara menggunakan API invoke_model_with_response_stream milik Amazon Bedrock untuk melakukan <i>stream output</i> model agar pengguna dapat menggunakan <i>output </i>saat dihasilkan. Dibandingkan menghasilkan <i>output </i>penuh sekaligus, API ini mengembalikan ResponseStream yang mengirimkan potongan <i>output </i>yang lebih kecil dari model saat diproduksi. Anda dapat menampilkan <i>output streaming </i>ini dalam tampilan yang berkelanjutan dan dapat digunakan.

In [ ]:
#invoke model with response stream
modelId = 'amazon.titan-text-premier-v1:0'
response = bedrock_client.invoke_model_with_response_stream(body=body, modelId=modelId, accept=accept, contentType=contentType)
stream = response.get('body')
output = list(stream)
output

In [ ]:
from IPython.display import display_markdown,Markdown,clear_output

In [ ]:
modelId = 'amazon.titan-text-premier-v1:0'
response = bedrock_client.invoke_model_with_response_stream(body=body, modelId=modelId, accept=accept, contentType=contentType)
stream = response.get('body')
output = []
i = 1
if stream:
    for event in stream:
        chunk = event.get('chunk')
        if chunk:
            chunk_obj = json.loads(chunk.get('bytes').decode())
            text = chunk_obj['outputText']
            clear_output(wait=True)
            output.append(text)
            display_markdown(Markdown(''.join(output)))
            i+=1

Kini Anda telah bereksperimen menggunakan SDK boto3 untuk mengakses API Amazon Bedrock. SDK ini menyediakan akses terprogram dasar ke kemampuan Bedrock. Dengan memanfaatkan API ini, Anda dapat menerapkan dua kasus penggunaan: 1) Membuat seluruh ringkasan teks konten berita AWS sekaligus, dan 2) <i>Streaming output</i> ringkasan dalam potongan untuk pemrosesan tambahan.

### Cobalah sendiri
- Ubah perintah untuk kasus penggunaan spesifik Anda dan evaluasi <i>output</i> model yang berbeda.
- Atur panjang token untuk memahami latensi dan responsivitas layanan.
- Terapkan berbagai prinsip rekayasa perintah untuk mendapatkan <i>output</i> yang lebih baik.

### Pembersihan

Anda telah menyelesaikan <i>notebook</i> ini. Untuk pindah ke bagian lab berikutnya, lakukan hal berikut:

- Tutup <i>file notebook</i> ini dan lanjutkan dengan **Task2b.ipynb**.